In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import t5

/home/ubuntu/tf-nvidia-3.7/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
!rm -rf out
!mkdir out

In [4]:
directory = 'out'

In [5]:
!ls t5-tiny-noisy-ms-en

checkpoint
events.out.tfevents.1657085426.huseincomel-desktop
events.out.tfevents.1657095770.huseincomel-desktop
graph.pbtxt
model.ckpt-1031000.data-00000-of-00002
model.ckpt-1031000.data-00001-of-00002
model.ckpt-1031000.index
model.ckpt-1031000.meta
model.ckpt-1073000.data-00000-of-00002
model.ckpt-1073000.data-00001-of-00002
model.ckpt-1073000.index
model.ckpt-1073000.meta
model.ckpt-1103000.data-00000-of-00002
model.ckpt-1103000.data-00001-of-00002
model.ckpt-1103000.index
model.ckpt-1103000.meta
model.ckpt-1132000.data-00000-of-00002
model.ckpt-1132000.data-00001-of-00002
model.ckpt-1132000.index
model.ckpt-1132000.meta
model.ckpt-1158000.data-00000-of-00002
model.ckpt-1158000.data-00001-of-00002
model.ckpt-1158000.index
model.ckpt-1158000.meta
model.ckpt-1188000.data-00000-of-00002
model.ckpt-1188000.data-00001-of-00002
model.ckpt-1188000.index
model.ckpt-1188000.meta
model.ckpt-1219000.data-00000-of-00002
model.ckpt-1219000.data-00001-of-00002
model.ckpt-1219000.index
model.ckpt

In [6]:
!cp t5-tiny-noisy-ms-en/model.ckpt-1402000* out
!cp t5-tiny-social-media/operative_config.gin out/

In [7]:
with open('out/checkpoint', 'w') as fopen:
    fopen.write('model_checkpoint_path: "model.ckpt-1402000"')

In [8]:
model = t5.models.MtfModel(
    model_dir=directory,
    tpu=None,
    tpu_topology=None,
    model_parallelism=1,
    batch_size=1,
    sequence_length={"inputs": 256, "targets": 256},
    learning_rate_schedule=0.003,
    save_checkpoints_steps=5000,
    keep_checkpoint_max=3,
    iterations_per_loop=100,
    mesh_shape="model:1,batch:1", 
    mesh_devices=["cpu:0"]
)


In [9]:
!rm -rf output/*

In [10]:
import gin

from t5.data import sentencepiece_vocabulary

DEFAULT_SPM_PATH = 'sp10m.cased.ms-en.model'
DEFAULT_EXTRA_IDS = 100
model_dir = directory

def get_default_vocabulary():
    return sentencepiece_vocabulary.SentencePieceVocabulary(
      DEFAULT_SPM_PATH, DEFAULT_EXTRA_IDS)

with gin.unlock_config():
    gin.parse_config_file(t5.models.mtf_model._operative_config_path(model_dir))
    gin.bind_parameter("Bitransformer.decode.beam_size", 1)
    gin.bind_parameter("Bitransformer.decode.temperature", 0)
    gin.bind_parameter("utils.get_variable_dtype.slice_dtype", "float32")
    gin.bind_parameter(
        "utils.get_variable_dtype.activation_dtype", "float32")
    
vocabulary = t5.data.SentencePieceVocabulary(DEFAULT_SPM_PATH)
estimator = model.estimator(vocabulary, disable_tpu=True)

INFO:tensorflow:Using config: {'_model_dir': 'out', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': None, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=100, num_shards=None, num_cores_per_replica=1, per_host_input_for_training=4, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None, eval_training_input_confi

In [11]:
import os

checkpoint_step = t5.models.mtf_model._get_latest_checkpoint_from_dir(model_dir)
model_ckpt = "model.ckpt-" + str(checkpoint_step)
checkpoint_path = os.path.join(model_dir, model_ckpt)
checkpoint_step, model_ckpt, checkpoint_path

(1402000, 'model.ckpt-1402000', 'out/model.ckpt-1402000')

In [12]:
from mesh_tensorflow.transformer import dataset as transformer_dataset

def serving_input_fn():
    inputs = tf.placeholder(
            dtype=tf.string,
            shape=[None],
            name="inputs")

    batch_size = tf.shape(inputs)[0]
    padded_inputs = tf.pad(inputs, [(0, tf.mod(-tf.size(inputs), batch_size))])
    dataset = tf.data.Dataset.from_tensor_slices(padded_inputs)
    dataset = dataset.map(lambda x: {"inputs": x})
    dataset = transformer_dataset.encode_all_features(dataset, vocabulary)
    dataset = transformer_dataset.pack_or_pad(
        dataset=dataset,
        length=model._sequence_length,
        pack=False,
        feature_keys=["inputs"]
    )
    dataset = dataset.batch(tf.cast(batch_size, tf.int64))
    features = tf.data.experimental.get_single_element(dataset)
    return tf.estimator.export.ServingInputReceiver(
        features=features, receiver_tensors=inputs)

out = estimator.export_saved_model('output', serving_input_fn, checkpoint_path=checkpoint_path)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:feature inputs : Tensor("Reshape:0", shape=(1, 256), dtype=int32)
INFO:tensorflow:Variable decoder/block_000/layer_000/SelfAttention/k                  size 294912       slice_size 294912       Shape[d_model=384, heads=768]                               
INFO:tensorflow:Variable decoder/block_000/layer_000/SelfAttention/o                  size 294912       slice_size 294912       Shape[heads=768, d_model=384]                               
INFO:tensorflow:Variable decoder/block_000/layer_000/SelfAttention/q                  size 294912       slice_size 294912       Shape[d_model=384, heads=768]                               
INFO:tensorflow:Variable decoder/block_000/layer_000/SelfAttention/relative_attention_bias size 384          slice_size 384          Shape[heads=12, buckets=32]                                 
INF

INFO:tensorflow:Variable decoder/block_002/layer_001/layer_norm/scale                 size 384          slice_size 384          Shape[d_model=384]                                          
INFO:tensorflow:Variable decoder/block_002/layer_002/DenseReluDense/wi/kernel         size 516096       slice_size 516096       Shape[d_model=384, d_ff=1344]                               
INFO:tensorflow:Variable decoder/block_002/layer_002/DenseReluDense/wo/kernel         size 516096       slice_size 516096       Shape[d_ff=1344, d_model=384]                               
INFO:tensorflow:Variable decoder/block_002/layer_002/layer_norm/scale                 size 384          slice_size 384          Shape[d_model=384]                                          
INFO:tensorflow:Variable decoder/block_003/layer_000/SelfAttention/k                  size 294912       slice_size 294912       Shape[d_model=384, heads=768]                               
INFO:tensorflow:Variable decoder/block_003/layer_000/Se

INFO:tensorflow:Variable encoder/block_003/layer_000/SelfAttention/o                  size 294912       slice_size 294912       Shape[heads=768, d_model=384]                               
INFO:tensorflow:Variable encoder/block_003/layer_000/SelfAttention/q                  size 294912       slice_size 294912       Shape[d_model=384, heads=768]                               
INFO:tensorflow:Variable encoder/block_003/layer_000/SelfAttention/v                  size 294912       slice_size 294912       Shape[d_model=384, heads=768]                               
INFO:tensorflow:Variable encoder/block_003/layer_000/layer_norm/scale                 size 384          slice_size 384          Shape[d_model=384]                                          
INFO:tensorflow:Variable encoder/block_003/layer_001/DenseReluDense/wi/kernel         size 516096       slice_size 516096       Shape[d_model=384, d_ff=1344]                               
INFO:tensorflow:Variable encoder/block_003/layer_001/De

2022-07-07 16:40:55.826721: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2022-07-07 16:40:55.833747: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2496000000 Hz
2022-07-07 16:40:55.834072: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x561fbeaeeb40 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-07-07 16:40:55.834093: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: output/temp-b'1657183253'/saved_model.pb


In [13]:
config = tf.ConfigProto()
config.allow_soft_placement = True
sess = tf.Session(config = config)
meta_graph_def = tf.saved_model.loader.load(
        sess,
        [tf.saved_model.tag_constants.SERVING],
        out)





Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from output/1657183253/variables/variables


2022-07-07 16:40:57.434965: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 49348608 exceeds 10% of system memory.


In [14]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'out/model.ckpt')

'out/model.ckpt'

In [15]:
strings = [
    n.name
    for n in tf.get_default_graph().as_graph_def().node
    if ('encoder' in n.op
    or 'decoder' in n.name
    or 'shared' in n.name
    or 'inputs' in n.name
    or 'output' in n.name
    or 'SentenceTokenizer' in n.name
    or 'self/Softmax' in n.name)
    and 'adam' not in n.name
    and 'Assign' not in n.name
]

In [16]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names,
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [17]:
freeze_graph('out', strings)



INFO:tensorflow:Restoring parameters from out/model.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


2022-07-07 16:40:58.973366: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 49348608 exceeds 10% of system memory.


INFO:tensorflow:Froze 144 variables.
INFO:tensorflow:Converted 144 variables to const ops.

4841 ops in the final graph.


In [18]:
import struct

unknown = b'\xff\xff\xff\xff'

def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
    for node in graph_def.node:
        
        if node.op == 'RefSwitch':
          node.op = 'Switch'
          for index in xrange(len(node.input)):
            if 'moving_' in node.input[index]:
              node.input[index] = node.input[index] + '/read'
        elif node.op == 'AssignSub':
          node.op = 'Sub'
          if 'use_locking' in node.attr: del node.attr['use_locking']
        elif node.op == 'AssignAdd':
          node.op = 'Add'
          if 'use_locking' in node.attr: del node.attr['use_locking']
        elif node.op == 'Assign':
          node.op = 'Identity'
          if 'use_locking' in node.attr: del node.attr['use_locking']
          if 'validate_shape' in node.attr: del node.attr['validate_shape']
          if len(node.input) == 2:
            node.input[0] = node.input[1]
            del node.input[1]
            
        if 'Reshape/shape' in node.name or 'Reshape_1/shape' in node.name:
            b = node.attr['value'].tensor.tensor_content
            arr_int = [int.from_bytes(b[i:i + 4], 'little') for i in range(0, len(b), 4)]
            if len(arr_int):
                arr_byte = [unknown] + [struct.pack('<i', i) for i in arr_int[1:]]
                arr_byte = b''.join(arr_byte)
                node.attr['value'].tensor.tensor_content = arr_byte
            
            if len(node.attr['value'].tensor.int_val):
                node.attr['value'].tensor.int_val[0] = -1
    
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [19]:
g = load_graph('out/frozen_model.pb')

In [28]:
i = g.get_tensor_by_name('import/inputs:0')
o = g.get_tensor_by_name('import/SelectV2_3:0')
i, o

(<tf.Tensor 'import/inputs:0' shape=(?,) dtype=string>,
 <tf.Tensor 'import/SelectV2_3:0' shape=(?, 256) dtype=int32>)

In [21]:
test_sess = tf.Session(graph = g)

In [23]:
%%time

o_ = test_sess.run(o, feed_dict = {i: [
    'terjemah Melayu ke Inggeris: i like u',
    'terjemah Melayu ke Inggeris: hidup ini',
    'terjemah Melayu ke Inggeris: ak tak paham la.',
]})
o_.shape

CPU times: user 2.14 s, sys: 96.6 ms, total: 2.23 s
Wall time: 261 ms


(3, 256)

In [24]:
import sentencepiece as spm
sp_model = spm.SentencePieceProcessor()
sp_model.Load(DEFAULT_SPM_PATH)

True

In [25]:
for k in range(len(o_)):
    print(k, sp_model.DecodeIds(o_[k].tolist()))

0 I'm like u
1 life
2 I don't understand it.


In [26]:
with open('ms-en-right.test') as fopen:
    right = fopen.read().split('\n')
    
with open('msd-en-left.test') as fopen:
    left = fopen.read().split('\n')

In [35]:
from tqdm import tqdm

batch_size = 16

results = []
for k in tqdm(range(0, len(left), batch_size)):
    inputs = [f'terjemah Melayu ke Inggeris: {s}' for s in left[k:k + batch_size]]
    o_ = test_sess.run(o, feed_dict = {i: inputs})
    for k in range(len(o_)):
        results.append(sp_model.DecodeIds(o_[k].tolist()))

  1%|▌                                                                               | 37/5475 [02:46<6:47:40,  4.50s/it]


KeyboardInterrupt: 

In [30]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU()

In [31]:
filtered_left, filtered_right = [], []
for no, r in enumerate(results):
    if len(r):
        filtered_left.append(r)
        filtered_right.append(right[no])

In [32]:
refs = [filtered_right]
sys = filtered_left

In [33]:
r = bleu.corpus_score(sys, refs)

In [34]:
r.__dict__

{'name': 'BLEU',
 'score': 64.34325697204099,
 '_mean': -1.0,
 '_ci': -1.0,
 '_verbose': '83.0/67.2/60.2/54.2 (BP = 0.985 ratio = 0.985 hyp_len = 336 ref_len = 341)',
 'bp': 0.9852292218149133,
 'counts': [279, 215, 183, 156],
 'totals': [336, 320, 304, 288],
 'sys_len': 336,
 'ref_len': 341,
 'precisions': [83.03571428571429,
  67.1875,
  60.19736842105263,
  54.166666666666664],
 'prec_str': '83.0/67.2/60.2/54.2',
 'ratio': 0.9853372434017595}